In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import plotly as py
import plotly.graph_objs as go

import warnings
warnings.filterwarnings('ignore')

from sklearn import preprocessing
import scipy.cluster.hierarchy as sch
from sklearn.cluster import AgglomerativeClustering

In [ ]:
train = pd.read_csv('/content/drive/MyDrive/240 Data Mining/Project/train 2.csv')
test = pd.read_csv('/content/drive/MyDrive/240 Data Mining/Project/test.csv')

In [ ]:
train.head()

,user,sku,category,query,click_time,query_time
0,000000df17cd56a5df4a94074e133c9d4739fae3,2125233,abcat0101001,Televisiones Panasonic 50 pulgadas,2011-09-01 23:44:52.533,2011-09-01 23:43:59.752
1,000001928162247ffaf63185cd8b2a244c78e7c6,2009324,abcat0101001,Sharp,2011-09-05 12:25:37.42,2011-09-05 12:25:01.187
2,000017f79c2b5da56721f22f9fdd726b13daf8e8,1517163,pcmcat193100050014,nook,2011-08-24 12:56:58.91,2011-08-24 12:55:13.012
3,000017f79c2b5da56721f22f9fdd726b13daf8e8,2877125,abcat0101001,rca,2011-10-25 07:18:14.722,2011-10-25 07:16:51.759
4,000017f79c2b5da56721f22f9fdd726b13daf8e8,2877134,abcat0101005,rca,2011-10-25 07:19:51.697,2011-10-25 07:16:51.759


In [ ]:
train.shape

(1865269, 6)

In [ ]:
train.head()

,user,sku,category,query,click_time,query_time
0,000000df17cd56a5df4a94074e133c9d4739fae3,2125233,abcat0101001,Televisiones Panasonic 50 pulgadas,2011-09-01 23:44:52.533,2011-09-01 23:43:59.752
1,000001928162247ffaf63185cd8b2a244c78e7c6,2009324,abcat0101001,Sharp,2011-09-05 12:25:37.42,2011-09-05 12:25:01.187
2,000017f79c2b5da56721f22f9fdd726b13daf8e8,1517163,pcmcat193100050014,nook,2011-08-24 12:56:58.91,2011-08-24 12:55:13.012
3,000017f79c2b5da56721f22f9fdd726b13daf8e8,2877125,abcat0101001,rca,2011-10-25 07:18:14.722,2011-10-25 07:16:51.759
4,000017f79c2b5da56721f22f9fdd726b13daf8e8,2877134,abcat0101005,rca,2011-10-25 07:19:51.697,2011-10-25 07:16:51.759


In [ ]:
train=train.drop(['click_time'],axis=1)

In [ ]:
train=train.drop(['query_time'],axis=1)

In [ ]:
train=train.drop(['query'],axis=1)
train=train.drop(['sku'],axis=1)

In [ ]:
train.shape

(1865269, 2)

In [ ]:
x=train

In [ ]:
label_encoder = preprocessing.LabelEncoder()

train['user'] = label_encoder.fit_transform(train['user'])
train['category'] = label_encoder.fit_transform(train['category'])
train.head()

,user,category
0,0,6
1,1,6
2,2,954
3,2,6
4,2,8


In [ ]:
users = train.set_index('user')['category'].rename('users')
display(users.head(10))
type(users)

user
0       6
1       6
2     954
2       6
2       8
3     652
4    1439
5     836
6      32
7     917
Name: users, dtype: int64

pandas.core.series.Series

In [ ]:
import sys
def size(obj):
    return "{0:.2f} MB".format(sys.getsizeof(obj) / (1000 * 1000))

In [ ]:
print('dimensions: {0};   size: {1};   unique_users: {2};   unique_categories: {3}'
      .format(users.shape, size(users), len(users.index.unique()), len(users.value_counts())))

dimensions: (1865269,);   size: 29.84 MB;   unique_users: 1268702;   unique_categories: 1540


In [ ]:
import numpy as np
from itertools import combinations, groupby
from collections import Counter

In [ ]:
# Returns frequency counts for items and item pairs
def freq(iterable):
    if type(iterable) == pd.core.series.Series:
        return iterable.value_counts().rename("freq")
    else:
        return pd.Series(Counter(iterable)).rename("freq")


# Returns number of unique orders
def order_count(order_item):
    return len(set(order_item.index))


# Returns generator that yields item pairs, one at a time
def get_item_pairs(order_item):
    order_item = order_item.reset_index().to_numpy()
    for order_id, order_object in groupby(order_item, lambda x: x[0]):
        item_list = [item[1] for item in order_object]

        for item_pair in combinations(item_list, 2):
            yield item_pair


# Returns frequency and support associated with item
def merge_item_stats(item_pairs, item_stats):
    return (item_pairs
                .merge(item_stats.rename(columns={'freq': 'freqA', 'support': 'supportA'}), left_on='Category_A', right_index=True)
                .merge(item_stats.rename(columns={'freq': 'freqB', 'support': 'supportB'}), left_on='Category_B', right_index=True))


# Returns name associated with item
def merge_item_name(rules, item_name):
    columns = ['Category_A','Category_B','freqAB','supportAB','freqA','supportA','freqB','supportB',
               'confidenceAtoB','confidenceBtoA','lift']
    rules = (rules
                .merge(item_name.rename(columns={'item_name': 'Category_A'}), left_on='Category_A', right_on='category')
                .merge(item_name.rename(columns={'item_name': 'Category_B'}), left_on='Category_B', right_on='category'))
    return rules[columns]

In [ ]:
train.sample

<bound method NDFrame.sample of             user  category
0              0         6
1              1         6
2              2       954
3              2         6
4              2         8
...          ...       ...
1865264  1268697      1438
1865265  1268698      1112
1865266  1268699      1449
1865267  1268700      1048
1865268  1268701       899

[1865269 rows x 2 columns]>

In [ ]:
def association_rules(order_item, min_support):

    print("Starting category_users: {:22d}".format(len(order_item)))


    # Calculate item frequency and support
    item_stats             = freq(order_item).to_frame("freq")
    item_stats['support']  = item_stats['freq'] / order_count(order_item) * 100


    # Filter from order_item items below min support
    qualifying_items       = item_stats[item_stats['support'] >= min_support].index
    order_item             = order_item[order_item.isin(qualifying_items)]

    print("Categories with support >= {}: {:15d}".format(min_support, len(qualifying_items)))
    print("Remaining category_users: {:21d}".format(len(order_item)))


    # Filter from order_item orders with less than 2 items
    order_size             = freq(order_item.index)
    qualifying_orders      = order_size[order_size >= 2].index
    order_item             = order_item[order_item.index.isin(qualifying_orders)]

    print("Remaining categories with 2+ users: {:11d}".format(len(qualifying_orders)))
    print("Remaining category_users: {:21d}".format(len(order_item)))


    # Recalculate item frequency and support
    item_stats             = freq(order_item).to_frame("freq")
    item_stats['support']  = item_stats['freq'] / order_count(order_item) * 100


    # Get item pairs generator
    item_pair_gen          = get_item_pairs(order_item)


    # Calculate item pair frequency and support
    item_pairs              = freq(item_pair_gen).to_frame("freqAB")
    item_pairs['supportAB'] = item_pairs['freqAB'] / len(qualifying_orders) * 100

    print("Category pairs: {:31d}".format(len(item_pairs)))


    # Filter from item_pairs those below min support
    item_pairs              = item_pairs[item_pairs['supportAB'] >= min_support]

    print("Category pairs with support >= {}: {:10d}\n".format(min_support, len(item_pairs)))


    # Create table of association rules and compute relevant metrics
    item_pairs = item_pairs.reset_index().rename(columns={'level_0': 'Category_A', 'level_1': 'Category_B'})
    item_pairs = merge_item_stats(item_pairs, item_stats)

    item_pairs['confidenceAtoB'] = item_pairs['supportAB'] / item_pairs['supportA']
    item_pairs['confidenceBtoA'] = item_pairs['supportAB'] / item_pairs['supportB']
    item_pairs['lift']           = item_pairs['supportAB'] / (item_pairs['supportA'] * item_pairs['supportB'])


    # Return association rules sorted by lift in descending order
    return item_pairs.sort_values('lift', ascending=False)


In [ ]:
%%time
rules = association_rules(users, 0.02)

Starting category_users:                1865269
Categories with support >= 0.02:             612
Remaining category_users:               1808527
Remaining categories with 2+ users:      265008
Remaining category_users:                832961
Category pairs:                          196424
Category pairs with support >= 0.02:       4107

CPU times: user 2.67 s, sys: 98.4 ms, total: 2.77 s
Wall time: 2.77 s


In [ ]:
rules

,Category_A,Category_B,freqAB,supportAB,freqA,supportA,freqB,supportB,confidenceAtoB,confidenceBtoA,lift
3828,419,419,102,0.038489,166,0.062640,166,0.062640,0.614458,0.614458,9.809412
2067,699,699,96,0.036225,180,0.067922,180,0.067922,0.533333,0.533333,7.852089
3806,1402,1402,96,0.036225,188,0.070941,188,0.070941,0.510638,0.510638,7.198044
3509,90,90,82,0.030942,177,0.066790,177,0.066790,0.463277,0.463277,6.936275
3868,712,712,58,0.021886,158,0.059621,158,0.059621,0.367089,0.367089,6.157052
...,...,...,...,...,...,...,...,...,...,...,...
916,471,1403,57,0.021509,81211,30.644735,2902,1.095061,0.000702,0.019642,0.000641
2933,105,1048,56,0.021131,7902,2.981797,29490,11.127966,0.007087,0.001899,0.000637
4011,465,1048,64,0.024150,10021,3.781395,29490,11.127966,0.006387,0.002170,0.000574
239,1048,610,111,0.041886,29490,11.127966,18253,6.887717,0.003764,0.006081,0.000546


In [ ]:
train.head()

,user,category
0,0,6
1,1,6
2,2,954
3,2,6
4,2,8


In [ ]:
rules_final = merge_item_name(rules, x).sort_values('lift', ascending=False)
display(rules_final)